In [3]:
!pip install moviepy
!pip install opencv-python
!pip install numpy
# googletrans
!pip install googletrans==4.0.0rc

In [1]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip

from googletrans import Translator
from langdetect import detect
translator = Translator()
import re
import codecs

In [2]:
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\\magick.exe"})

In [14]:
# 자막 파일 경로
srt_path = "C:/Users/hjn/Downloads/subtitles_out33.srt"
# 동영상 파일 경로
video_path = "C:/Users/hjn/Downloads/input.avi"
# 동영상 로드
video = VideoFileClip(video_path)
# 동영상 추출
#output_path="C:/Users/ella6/Videos/output6.mp4"
# 원하는 추출 언어
output_lang = 'ko'
# 얼굴 pickle 파일
face_pickle = "C:/Users/hjn/Downloads/faces.pckl"

In [15]:
import pickle

# pickle 파일 읽기
with open(face_pickle, 'rb') as f:
    faces = pickle.load(f)
    
#with open(srt_path, "r") as f:
#    lines = f.readlines()

# srt 파일 읽기
with codecs.open(srt_path, 'r', 'utf-8') as file:
    lines = file.readlines()

['WEBVTT\r\n', '\r\n', '1\r\n', '00:00:04.300 --> 00:00:06.869\r\n', 'Appears to be a standard sequence violation\r\n', '\r\n', '2\r\n', '00:00:07.530 --> 00:00:12.500\r\n', 'branch is growing at a stable rate and slope variant identified.\r\n', '\r\n', '3\r\n', '00:00:13.630 --> 00:00:14.600\r\n', 'I beg your pardon.\r\n', '\r\n', '4\r\n', '00:00:14.689 --> 00:00:17.430\r\n', 'On behalf of the time variance authority,\r\n', '\r\n', '5\r\n', '00:00:17.440 --> 00:00:20.620\r\n', 'I hereby arrest you for crimes against the sacred timeline.\r\n', '\r\n', '6\r\n', '00:00:21.170 --> 00:00:22.079\r\n', 'Hands up.\r\n', '\r\n', '7\r\n', '00:00:23.110 --> 00:00:24.540\r\n', "You're coming with us.\r\n", '\r\n', '8\r\n', '00:00:25.409 --> 00:00:26.860\r\n', "I'm sorry. Who's us?\r\n", '\r\n', '9\r\n', '00:00:28.739 --> 00:00:30.409\r\n', 'Last chance variant?\r\n', '\r\n', '10\r\n', '00:00:34.479 --> 00:00:35.950\r\n', "It's been a very long day\r\n", '\r\n', '11\r\n', '00:00:36.369 --> 00:00:3

In [16]:
# srt파일의 시간을 초단위로 변경하는 함수
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    seconds = h * 3600 + m * 60 + s
    return seconds

# 번역함수
def translate(text, target='ko'):
    # do not translate if text is Korean.
    if detect(text) == target:
        return text

    result = translator.translate(text, str(target)).text
    return result

In [22]:
# 자막이 배치된 리스트
subtitles_clip = []

for i in range(0, len(lines)-2, 4):
    # aws이면 앞의 두줄 삭제
    line_2 = lines[2:]
    # srt파일에서 자막의 시작시간과 끝 시간을 뽑아주는 함수
    start_time, end_time = line_2[i + 1].strip().replace('\r','').split(' --> ')
    text = line_2[i + 2].strip()
    
    # 자막 생성 안되었을 경우 에러 방지
    if text == '':
        continue
        
    # 원하는 언어로 자막 번역
    text = translate(text, str(output_lang))
    # 자막을 배치
    text_clip = TextClip(text, fontsize=24, color='white', font='Arial-Unicode-MS')\
    .set_position(("center", "bottom")).set_start(start_time).set_end(end_time)
    
    start = int(time_to_seconds(start_time.split('.')[0])*video.fps)
    end = int(time_to_seconds(end_time.split('.')[0])*video.fps)
    mid = int((start+end)//2)
    # 자막의 중간시간 기준으로 나타나는 bounding box가 없을때 아래쪽에 자막 배치
    if [d['bbox'] for d in faces[mid]] == []:
        text_clip = text_clip.set_position(("center", "bottom"))
        text_clip = text_clip.set_start(start_time).set_end(end_time)
    # 자막의 중간시간 기준에 나타나는 bounding box 기준으로 자막 배치
    else:
        # bounding box의 왼쪽 아래 모서리부분에 자막배치
        x, y = [d['bbox'] for d in faces[mid]][0][0], [d['bbox'] for d in faces[mid]][0][3]
        text_clip = text_clip.set_position([x, y])
        text_clip = text_clip.set_start(start_time).set_end(end_time)
    subtitles_clip.append(text_clip)

In [7]:
# 텍스트 클립들을 동영상 위에 합성
video_with_text = CompositeVideoClip([video] + subtitles_clip)

In [8]:
# 합성된 동영상을 저장
output_path="C:/Users/hjn/Downloads/moviepyvideos/output29.mp4"
video_with_text.write_videofile(output_path, codec='libx264', audio_codec='aac')

Moviepy - Building video C:/Users/hjn/Downloads/moviepyvideos/output29.mp4.
MoviePy - Writing audio in output29TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video C:/Users/hjn/Downloads/moviepyvideos/output29.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/hjn/Downloads/moviepyvideos/output29.mp4
